In [1]:
from __future__ import print_function, division

import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Flatten, Reshape
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers import concatenate
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Conv2DTranspose
from keras.layers import Concatenate

import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional, concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, UpSampling2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Lambda
from keras import backend as K
import numpy as np
from keras.layers import Softmax

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [15]:
samples = np.load("sample_2019-11-13-16-10.npy")

In [160]:
samples[0]

array([[ 0. ,  0.5,  0. , ...,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 9.5, 10. ,  0. , ...,  0. ,  0. ,  0. ],
       [10. , 10.5,  0. , ...,  0. ,  0. ,  0. ],
       [10.5, 11. ,  0. , ...,  0. ,  0. ,  0. ]])

In [9]:
def get_notes():
    notes=np.load('sample_2019-11-13-16-10.npy')
    return notes

notes = get_notes()
print(notes.shape)

def make_pitch_generator():
    model = Sequential()
    dropout = 0.4
    cols = 32
    rows = 5
    depth = 64 + 64 + 64 + 64
    # Out: 20 X 130
    model.add(Dense(rows*cols, input_dim=100))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))
    model.add(Reshape((rows, cols, 1)))
    model.add(Dropout(dropout))
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(1, 5, padding='same'))
    model.add(Softmax(axis=1))
    model.summary()
    return model

generator_pitch = make_pitch_generator()

def make_generator_start_end():
    model = Sequential()
    model.add(Dense(20, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(20))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(40))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Reshape((20,2,1)))
    model.summary()
    return model

generator_start_end = make_generator_start_end()

generator_concat = concatenate([generator_pitch.output, generator_start_end.output], axis=-2)
generator_concat = Dense(1)(generator_concat)
generator = Model(inputs=[generator_pitch.input, generator_start_end.input], outputs=generator_concat)

(100000, 20, 130)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 160)               16160     
_________________________________________________________________
batch_normalization_8 (Batch (None, 160)               640       
_________________________________________________________________
activation_5 (Activation)    (None, 160)               0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 5, 32, 1)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 32, 1)          0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 10, 64, 1)         0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 10, 64, 128)       332

In [10]:
generator.output

<tf.Tensor 'dense_10/BiasAdd:0' shape=(?, 20, 130, 1) dtype=float32>

In [11]:
noise = np.random.uniform(-1.0, 1.0, size=[50, 100])

In [14]:
result = generator.predict([noise, noise])

In [21]:
0.05401744 * 20

1.0803487999999999

In [38]:
np.sum(result[0][0:20, 5])

1.0719454

In [42]:
# generate noise
n_samples = 100
noise_length = 128
noise = np.random.uniform(-1.0, 1.0, size=[n_samples, noise_length])

In [55]:
# generator
num_nodes = 128
dropout_rate = 0.2

random_inputs = Input(shape=(noise_length,))
dense1 = Dense(num_nodes, activation='relu')(random_inputs)
dropout1 = Dropout(dropout_rate)(dense1)
batchNorm1 = BatchNormalization()(dropout1)
dense2 = Dense(num_nodes, activation='relu')(batchNorm1)
dropout2 = Dropout(dropout_rate)(dense2)
batchNorm2 = BatchNormalization()(dropout2)

pitch = Dense(20 * 128)(batchNorm2)
duration = Dense(20 * 2)(batchNorm2)

pitch_reshaped = Reshape((20, 128))(pitch)
duration_reshaped = Reshape((20, 2))(duration)

pitch_output = Softmax(axis=-1)(pitch_reshaped)
duration_output = Dense(2, activation='relu', name='duration')(duration_reshaped)
output_concat = concatenate([duration_output, pitch_output])

generator = Model(inputs=random_inputs, outputs=output_concat)

In [61]:
generator.predict(noise)

array([[[0.15645309, 0.        , 0.00710962, ..., 0.0073821 ,
         0.00714432, 0.00792802],
        [0.18877085, 0.19557388, 0.00731536, ..., 0.00807713,
         0.00728555, 0.00799938],
        [0.25478068, 0.10027927, 0.00714836, ..., 0.00811918,
         0.00765923, 0.00834646],
        ...,
        [0.        , 0.        , 0.00692928, ..., 0.00879381,
         0.00786162, 0.00867419],
        [0.07986229, 0.09001254, 0.00827463, ..., 0.00678803,
         0.00696803, 0.00791797],
        [0.        , 0.15378585, 0.00692377, ..., 0.00842176,
         0.00723738, 0.00742429]],

       [[0.14973535, 0.        , 0.00637885, ..., 0.00735982,
         0.00789291, 0.00798584],
        [0.5437106 , 0.18771492, 0.00795301, ..., 0.00752186,
         0.00752698, 0.0079512 ],
        [0.10416038, 0.09374883, 0.00750843, ..., 0.00789902,
         0.00814482, 0.00773575],
        ...,
        [0.        , 0.        , 0.00704564, ..., 0.0084009 ,
         0.00738068, 0.00742434],
        [0. 

In [237]:
# discriminator
num_nodes = 128
dropout_rate = 0.2

data_inputs = Input(shape=(20, 130, 1))
dense1 = Dense(num_nodes, activation='relu')(data_inputs)
dropout1 = Dropout(dropout_rate)(dense1)
batchNorm1 = BatchNormalization()(dropout1)

dense2 = Dense(num_nodes, activation='relu')(batchNorm1)
dropout2 = Dropout(dropout_rate)(dense2)
batchNorm2 = BatchNormalization()(dropout2)

true_fake_output = Dense(2, activation='sigmoid')(batchNorm2)

discriminator = Model(inputs=[data_inputs], outputs=[true_fake_output])

discriminator.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [263]:
generator

<tf.Tensor 'dense_155/BiasAdd:0' shape=(?, 20, 130, 1) dtype=float32>

In [ ]:
gan_input = Input(shape=(100,))
x = generator()
gan_output= discriminator(x)
gan= Model(inputs=gan_input, outputs=gan_output)

In [258]:
optimizer = RMSprop(lr=0.0001, decay=3e-8)
generator_concat = concatenate([generator1.output, generator2.output], axis=-2)
generator_concat = Dense(1)(model_concat)
adversarial = Model(inputs=[generator1.input, generator2.input], outputs=discriminator)
adversarial.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <keras.engine.training.Model object at 0x1cba229a20>

In [ ]:
def get_generator_model(num_elements):
    
    inputs = Input(shape=(num_elements,))
    dense1 = Dense()

In [ ]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist",\
        	one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows,\
        	self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()